In [ ]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from deepface import DeepFace
import time

# ============================
# CONFIG GENERAL
# ============================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

class_names = ["angry", "fear", "happy", "neutral", "sad", "surprise"]

IMG_SIZE = 224

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# ============================
# MODELO RESNET50 EMOCIONES
# ============================

def get_model(num_classes=6):
    base = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    num_ftrs = base.fc.in_features  # 2048

    base.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.BatchNorm1d(512),
        nn.Dropout(0.4),
        nn.Linear(512, num_classes)
    )

    for name, param in base.named_parameters():
        if 'layer1' in name or 'layer2' in name:
            param.requires_grad = False
        else:
            param.requires_grad = True

    return base

model = get_model(num_classes=len(class_names)).to(device)
state_dict = torch.load("best_model_emotions.pth", map_location=device)
model.load_state_dict(state_dict, strict=True)
model.eval()

@torch.no_grad()
def predict_emotion(face_bgr):
    face_rgb = cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB)
    tensor = transform(face_rgb)
    tensor = tensor.unsqueeze(0).to(device)
    logits = model(tensor)
    probs = F.softmax(logits, dim=1).cpu().numpy()[0]
    idx = int(np.argmax(probs))
    return class_names[idx], probs

# ============================
# FONDOS POR EMOCIÓN
# ============================

def draw_emotion_background(frame, label):
    h, w, _ = frame.shape
    overlay = np.zeros_like(frame)

    if label == "happy":
        for i in range(h):
            alpha = i / h
            color = (
                int(255 * (1 - alpha) + 200 * alpha),
                int(255 * (1 - alpha) + 220 * alpha),
                int(200 * (1 - alpha) + 150 * alpha)
            )
            overlay[i, :] = color

        center = (w - 100, 100)
        cv2.circle(overlay, center, 50, (0, 255, 255), -1)
        for angle in range(0, 360, 30):
            x2 = int(center[0] + 80 * np.cos(np.deg2rad(angle)))
            y2 = int(center[1] + 80 * np.sin(np.deg2rad(angle)))
            cv2.line(overlay, center, (x2, y2), (0, 255, 255), 3)
        for cx, cy in [(120, 120), (220, 80), (80, 80)]:
            cv2.ellipse(overlay, (cx, cy), (60, 30), 0, 0, 360, (255, 255, 255), -1)
            cv2.ellipse(overlay, (cx+30, cy+10), (50, 25), 0, 0, 360, (255, 255, 255), -1)

    elif label == "sad":
        for i in range(h):
            alpha = i / h
            color = (
                int(100 * (1 - alpha) + 50 * alpha),
                int(100 * (1 - alpha) + 80 * alpha),
                int(140 * (1 - alpha) + 160 * alpha)
            )
            overlay[i, :] = color

        for cx, cy in [(w//4, 80), (w//2, 60), (3*w//4, 90)]:
            cv2.ellipse(overlay, (cx, cy), (120, 40), 0, 0, 360, (80, 80, 90), -1)

        for x in range(0, w, 20):
            y_start = np.random.randint(0, 50)
            y_end = y_start + np.random.randint(30, 80)
            cv2.line(overlay, (x, y_start), (x+5, y_end), (200, 200, 255), 1)

        for _ in range(20):
            cx = np.random.randint(0, w)
            cy = np.random.randint(h//2, h)
            cv2.circle(overlay, (cx, cy), 3, (230, 230, 255), -1)

    elif label == "angry":
        for i in range(h):
            alpha = i / h
            color = (
                int(0 * (1 - alpha) + 0 * alpha),
                int(0 * (1 - alpha) + 50 * alpha),
                int(100 * (1 - alpha) + 255 * alpha)
            )
            overlay[i, :] = color

        for _ in range(80):
            base_x = np.random.randint(0, w)
            height = np.random.randint(40, 120)
            pts = np.array([
                (base_x, h),
                (base_x + np.random.randint(-20, 20), h - height),
                (base_x + np.random.randint(-40, 40), h)
            ], np.int32)
            cv2.fillConvexPoly(overlay, pts,
                               (0, np.random.randint(100, 180), 255))

    elif label == "fear":
        for i in range(h):
            alpha = i / h
            color = (
                int(40 * (1 - alpha) + 20 * alpha),
                int(40 * (1 - alpha) + 0 * alpha),
                int(80 * (1 - alpha) + 120 * alpha)
            )
            overlay[i, :] = color

        fog = overlay.copy()
        for _ in range(40):
            cx = np.random.randint(0, w)
            cy = np.random.randint(0, h)
            r = np.random.randint(30, 80)
            cv2.circle(fog, (cx, cy), r, (80, 80, 120), -1)
        cv2.addWeighted(fog, 0.3, overlay, 0.7, 0, overlay)

        center = (w//2, h//3)
        cv2.ellipse(overlay, center, (60, 25), 0, 0, 360, (200, 200, 220), 2)
        cv2.circle(overlay, center, 8, (200, 200, 220), -1)

    elif label == "surprise":
        overlay[:] = (30, 30, 30)
        center = (w//2, h//2)
        for angle in range(0, 360, 10):
            length = np.random.randint(80, 200)
            x2 = int(center[0] + length * np.cos(np.deg2rad(angle)))
            y2 = int(center[1] + length * np.sin(np.deg2rad(angle)))
            cv2.line(overlay, center, (x2, y2), (255, 255, 255), 1)
        for _ in range(80):
            rx = np.random.randint(0, w)
            ry = np.random.randint(0, h)
            color = (
                np.random.randint(200, 255),
                np.random.randint(200, 255),
                np.random.randint(200, 255)
            )
            cv2.circle(overlay, (rx, ry), 2, color, -1)

    else:  # neutral
        for i in range(h):
            alpha = i / h
            color = (
                int(160 * (1 - alpha) + 120 * alpha),
                int(160 * (1 - alpha) + 160 * alpha),
                int(170 * (1 - alpha) + 180 * alpha)
            )
            overlay[i, :] = color

    cv2.addWeighted(overlay, 0.9, frame, 0.1, 0, frame)

# ============================
# BUCLE WEBCAM + DEEPFACE (opencv)
# ============================

def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("No se pudo abrir la webcam")
        return

    print("Pulsa 'q' para salir")
    prev_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        orig_frame = frame.copy()

        label = "neutral"
        face_box = None

        # 1. Detección de cara con DeepFace (backend opencv) [web:103][web:95]
        try:
            detections = DeepFace.extract_faces(
                img_path=orig_frame,
                detector_backend="opencv",
                enforce_detection=False
            )
            if len(detections) > 0:
                det = detections[0]
                fa = det["facial_area"]
                x, y, w, h = fa["x"], fa["y"], fa["w"], fa["h"]

                # clamp a la imagen
                x = max(0, x)
                y = max(0, y)
                w = max(1, min(w, orig_frame.shape[1] - x))
                h = max(1, min(h, orig_frame.shape[0] - y))

                face_box = (x, y, w, h)
                face_bgr = orig_frame[y:y+h, x:x+w]
                label, probs = predict_emotion(face_bgr)
        except Exception:
            label = "neutral"
            face_box = None

        # 2. Pintar fondo según emoción
        draw_emotion_background(frame, label)

        # 3. Volver a poner la cara original encima del fondo
        if face_box is not None:
            x, y, w, h = face_box
            face_roi = orig_frame[y:y+h, x:x+w]
            frame[y:y+h, x:x+w] = face_roi
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255,255,255), 2)

        # 4. Texto emoción y FPS
        cv2.putText(frame, label, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2, cv2.LINE_AA)

        now = time.time()
        fps = 1.0 / (now - prev_time + 1e-6)
        prev_time = now
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 1, cv2.LINE_AA)

        cv2.imshow("Emotion Backgrounds (DeepFace OpenCV)", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Device: cpu
Pulsa 'q' para salir
